# Final Data Analysis

# United States Education and Occupation Information

This extracts the all information in the 2013-2017 American Community Survey PUMS Data into a pandas data frame. Then key variables are obtained to gather aggregate information on what are the wages (WAGP) of the people of the US based on their level of education (SCHL), field of degree (FOD1P) and occupation (OCCP).

# Variables Used:

Wages (WAGP,
Level of Education (SCHL),
Field of Degree (FOD1P),
Occupation (OCCP),
Sex (SEX),
Looking for Job (NWLK)

## Dependencies

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading & Organization of Data

The data is downloaded as a csv from US Census Bureau as a CSV.
Data can be downloaded here:
https://www2.census.gov/programs-surveys/acs/data/pums/2017/5-Year/
It is then read in to a Pandas DataFrame
The columns are coded according the 2013-2017 ACS PUMS Data Dictionary
Data Dictionary can be found here:
https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2013-2017.pdf?#

In [2]:
# Loading US Data csv files
file_to_load = ['data_files/psam_pusa.csv'
                ,'data_files/psam_pusb.csv'
                ,'data_files/psam_pusc.csv'
                ,'data_files/psam_pusd.csv']
chunksize = 100000

In [3]:
# Obtaining the desired columns from a Larger Dataset into smaller CSVs
count = 1

for path in file_to_load:
    degree_dataT = pd.read_csv(path, chunksize=chunksize, iterator = True)
    degree_data = pd.concat(degree_dataT, ignore_index=True)

    degree_data = degree_data[["FOD1P","OCCP","WAGP","SEX","SCHL","NWLK"]]

    degree_data.to_csv(f'data_files/usa_{count}.csv')
    count += 1
    %reset_selective -f degree

In [15]:
# Load US selected variables into CSVs and create a single DataFrame
usa_files = ['data_files/usa_1.csv',
            'data_files/usa_2.csv',
            'data_files/usa_3.csv',
            'data_files/usa_4.csv']

us_data1 = pd.read_csv(usa_files[0]).reset_index()
us_data2 = pd.read_csv(usa_files[1]).reset_index()
us_data3 = pd.read_csv(usa_files[2]).reset_index()
us_data4 = pd.read_csv(usa_files[3]).reset_index()

degree_data = pd.concat([us_data1,us_data2,us_data3,us_data4]).drop('index',axis = 1)
degree_data = degree_data.drop('Unnamed: 0',axis = 1)

degree_data = pd.DataFrame(degree_data)

# Removing NaN and 0 values from the wages.
degree_data.dropna(subset = ['WAGP'], inplace = True)
degree_data = degree_data[degree_data != 0]

In [16]:
# Getting the Occupation codes and names 
# obatined from 2013-2017 ACS PUMS Data Dictionary

occ_code = pd.read_csv("data_files/Occupation_codes.csv", header = None)
occ_code = pd.DataFrame(occ_code)
occ_code = occ_code.rename(columns = {0: "occupation_code",
                                     1: "occupation_field",
                                     2: "occupation_name"})
occ_code = occ_code.dropna(1)

## Retrieving Different Codes for Degree Types

In [17]:
# Here we are retrieving the degree codes and that correspond with the degree name. 
# Converted the information to a csv from the Data Dictionary using Excel

degree_code = pd.read_csv("data_files/degree_codes.csv", header = None)
degree_code = pd.DataFrame(degree_code)
degree_code = degree_code.rename(columns = {0: "degree_code",
                                            1: "degree_name"})
degree_code.head()

,degree_code,degree_name
0,1100.0,General Agriculture
1,1101.0,Agriculture Production And Management
2,1102.0,Agricultural Economics
3,1103.0,Animal Sciences
4,1104.0,Food Science


## Question 1 & 2: What are the 10 highest/lowest paying college degrees in the United States?

In [18]:
# Group by degree code
obs = degree_data.groupby('FOD1P')

# Count number of people for each degree
degree_counts = pd.DataFrame(obs.size())

# Get the mean wage for each degree
income = pd.DataFrame(obs.mean()['WAGP'])

# Merging of DataFrames
income_per_degree = degree_counts.merge(income, on = 'FOD1P')\
                    .reset_index()

# Renaming of DataFrames Columns
income_per_degree = income_per_degree.rename(columns = {"FOD1P": "degree_code",
                                                        0: "number_of_people",
                                                        "WAGP":"wage/salary"})

# income_per_degree = degree_code.merge(income_per_degree, on = 'degree_code')

# income_per_degree.head()
# Show 10 highest paying and 10 lowest paying degrees.
# print("These are the 10 highest paying degrees")
# income_per_degree.sort_values('wage/salary', ascending = False).head(10)
# print("\n##################################################################\n")
# print("These are the 10 lowest paying degrees")
# income_per_degree.sort_values('wage/salary', ascending = True).head(10)

## Question 3: Difference in Male/Female Wage Data

We are looking for the difference in men and women in specific degrees. We also look at the wage differences of each sex.

In [19]:
# Grouping degree information by sex
male = degree_data[degree_data['SEX'] == 1].groupby('FOD1P')
female = degree_data[degree_data['SEX'] == 2].groupby('FOD1P')

# Obtaining male and female mean wages per degree
male_wage = pd.DataFrame(male.mean()['WAGP'])
female_wage = pd.DataFrame(female.mean()['WAGP'])
                    
# Obtaining percentages of male and female per degree
male_count = pd.DataFrame(male.size())
male_perc = male_count / degree_counts * 100

female_count = pd.DataFrame(female.size())
female_perc = female_count / degree_counts * 100

# Merging data into one frame and renaming columns
sex_data = male_perc.merge(male_wage, on = 'FOD1P').reset_index()
sex_data = sex_data.merge(female_perc, on = 'FOD1P')
sex_data = sex_data.merge(female_wage, on = 'FOD1P')
sex_data = sex_data.merge(male_count, on = 'FOD1P')
sex_data = sex_data.merge(female_count, on = 'FOD1P')

sex_data.columns = ['degree_code',
                    'male_%',
                    'male_wage',
                    'female_%',
                    'female_wage',
                    'male_count',
                    'female_count']

sex_data = degree_code.merge(sex_data, on = 'degree_code')

# Cleaning of data
sex_data = sex_data.drop('degree_code',axis = 1)
sex_data = sex_data.round(2)

# Saving the data to a csv
sex_data.to_csv('data_to_plot/USSexDiffWages.csv', index = False)

# DataFrame that shows the college degrees where males are in the majority
# print("DataFrame that shows the college degrees where males are in the majority\n")
# print(sex_data.sort_values('male_%', ascending = False).head())

# print("\n#######################################################################################")

# # DataFrame that shows the college degrees where females are in the majority
# print("\nDataFrame that shows the college degrees where females are in the majority\n")
# print(sex_data.sort_values('male_%', ascending = True).head())
sex_data.head()

,degree_name,male_%,male_wage,female_%,female_wage,male_count,female_count
0,General Agriculture,79.15,69877.11,20.85,48233.40,8050,2120
1,Agriculture Production And Management,77.18,75170.88,22.82,49261.90,5997,1773
2,Agricultural Economics,82.03,95632.87,17.97,60986.32,2761,605
3,Animal Sciences,50.99,81673.46,49.01,49959.10,5279,5073
4,Food Science,42.80,94409.63,57.20,64545.54,1076,1438


## EXTRA DATA : People that are Looking for Jobs

In [20]:
# Grouping data by degree code only including people that are looking for a job
obs_looking = degree_data[degree_data['NWLK'] == 1].groupby('FOD1P')

# Getting counts of people for each degree and renaming column
looking_counts = pd.DataFrame(obs_looking.size()).reset_index()
looking_counts = looking_counts.rename(columns = {"FOD1P": "degree_code",
                                                  0: "looking"})
looking_counts.head()

,degree_code,looking
0,1100.0,163
1,1101.0,159
2,1102.0,49
3,1103.0,214
4,1104.0,86


## Question 4: How does the expected median salary of a particular degree compare with the average salary of the US?

In [21]:
# Find the average salary of United States
us_avg_salary = degree_data['WAGP'].dropna()
us_avg_salary = us_avg_salary[us_avg_salary != 0]
us_avg_salary = us_avg_salary.mean()

print("Average US salary is: $" + str(us_avg_salary.round()))

# Group by degree code
obs = degree_data.groupby('FOD1P')

# Getting mean wage for each degree
income = obs.mean()['WAGP']
income = pd.DataFrame(income)

# Merging counts and income frames
income_per_degree_comp = pd.DataFrame(income).reset_index()

# Renaming columns
income_per_degree_comp = income_per_degree_comp.rename(columns = {"FOD1P": "degree_code", "WAGP": "Wage/Salary"})

# income_per_degree_comp = degree_code.merge(income_per_degree_comp, on = 'degree_code')

# Will show 10 college degrees that earn less than the average US salary
income_per_degree_comp.sort_values('Wage/Salary').where(income_per_degree_comp['Wage/Salary'] <= us_avg_salary).head(10)

Average US salary is: $47016.0


,degree_code,Wage/Salary
31,2307.0,39445.804644
146,6099.0,43086.735905
28,2304.0,43532.344927
36,2312.0,44434.249125
145,6007.0,44888.510949
102,4901.0,45120.220694
141,6003.0,45352.637431
38,2314.0,45892.091218
124,5403.0,46250.141437
70,2901.0,46649.917623


## Question 5: People with college degrees working in fields that DO NOT require a college degree

In [22]:
# Gathered the keys for what are considered low_end/non-college jobs (Servers,telemarketers, etc.) 
# Here's our list of low-end jobs (obtained from 2013-2017 ACS PUMS Data Dictionary)

# 4020 .EAT-COOKS
# 4030 .EAT-FOOD PREPARATION WORKERS
# 4040 .EAT-BARTENDERS
# 4050 .EAT-COMBINED FOOD PREPARATION AND SERVING WORKERS,
# .INCLUDING FAST FOOD
# 4060 .EAT-COUNTER ATTENDANTS, CAFETERIA, FOOD CONCESSION, AND
# .COFFEE SHOP
# 4110 .EAT-WAITERS AND WAITRESSES
# 4120 .EAT-FOOD SERVERS, NONRESTAURANT
# 4130 .EAT-MISCELLANEOUS FOOD PREPARATION AND SERVING RELATED
# .WORKERS, INCLUDING DINING ROOM AND CAFETERIA ATTENDANTS AND
# .BARTENDER HELPERS
# 4140 .EAT-DISHWASHERS
# 4150 .EAT-HOSTS AND HOSTESSES, RESTAURANT, LOUNGE, AND COFFEE SHOP
# 4220 .CLN-JANITORS AND BUILDING CLEANERS
# 4230 .CLN-MAIDS AND HOUSEKEEPING CLEANERS
# 4240 .CLN-PEST CONTROL WORKERS
# 4250 .CLN-GROUNDS MAINTENANCE WORKERS
# 4720 .SAL-CASHIERS
# 4740 .SAL-COUNTER AND RENTAL CLERKS
# 4750 .SAL-PARTS SALESPERSONS
# 4760 .SAL-RETAIL SALESPERSONS
# 4940 .SAL-TELEMARKETERS

low_end_jobs = [4020,4030,4040,4050,4060,4110,4120,4130,4140,4150
                ,4220,4230,4240,4250,4720,4740,4750,4760,4940]

data_low_end = degree_data[degree_data["OCCP"].isin(low_end_jobs)]

low_end_counts = pd.DataFrame(data_low_end.groupby('FOD1P').size()).reset_index()
low_end_counts = low_end_counts.rename(columns = {0 : "non-degree jobs",
                                                  "FOD1P": "degree_code",})

# low_end_counts = degree_code.merge(low_end_counts, on = 'degree_code')

low_end_counts.sort_values('non-degree jobs', ascending = False).head()

,degree_code,non-degree jobs
161,6203.0,7986
158,6200.0,6276
114,5200.0,5968
73,3301.0,3926
164,6206.0,3917


## EXTRA DATA : Data on people who did not attend college

In [23]:
# Using the Data Dictionary, the a range of level of educations can be obtained

# 12 .Grade 9
# 13 .Grade 10
# 14 .Grade 11
# 15 .12th grade - no diploma
# 16 .Regular high school diploma
# 17 .GED or alternative credential

non_college = list(range(12,18))

# Getting all the data for people that are or have not been college students
data_non_college = degree_data[degree_data["SCHL"].isin(non_college)]

# Create dataframe
non_college_counts = pd.DataFrame(data_non_college.groupby('OCCP').size()).reset_index()
non_college_wage = pd.DataFrame(data_non_college.groupby('OCCP').mean())[["WAGP"]].reset_index()

non_college_data = non_college_counts.merge(non_college_wage, on = 'OCCP')
non_college_data = non_college_data.rename(columns = {0 : "non-college count",
                                                      "OCCP": "occupation_code",
                                                      "WAGP":"wage/salary"})
# Merging occupation codes with salary information
occ_full_data = occ_code.merge(non_college_data, on = "occupation_code")

# Cleaning of data
occ_full_data = occ_full_data.drop('occupation_code', axis = 1)
occ_full_data = occ_full_data.round(2)

# Saving the data to a csv
occ_full_data.to_csv('data_to_plot/USOccuWages.csv')

# Shows which individuals who did not attend college earn more than those with college degrees
occ_full_data.where(occ_full_data['wage/salary'] >= us_avg_salary).dropna().head()

,occupation_field,occupation_name,non-college count,wage/salary
0,MGR,General And Operations Managers,9557.0,66772.16
1,MGR,Advertising And Promotions Managers,179.0,57341.48
2,MGR,Marketing And Sales Managers,4459.0,64577.70
3,MGR,Public Relations And Fundraising Managers,114.0,63470.00
4,MGR,Administrative Services Managers,1746.0,55633.66


## Compilation of all US data into one DataFrame

In [24]:
#merge to a final dataframe including all data
full_data = degree_code.merge(income_per_degree, on = 'degree_code')
full_data = full_data.merge(low_end_counts, on = 'degree_code')
full_data = full_data.merge(looking_counts, on = "degree_code")

#finding percentages for people looking for jobs
full_data["looking_%"] = full_data["looking"]\
                                /full_data["number_of_people"]\
                                * 100
#finding percentages of people in non-degree jobs
full_data["NDJ_%"] = full_data["non-degree jobs"]\
                        /full_data["number_of_people"]\
                        * 100
#overall percentage for people looking for job
NDJ_count = full_data["non-degree jobs"].sum()\
                        /full_data["number_of_people"].sum()\
                        * 100
#cleaning data
full_data = full_data.drop('degree_code',axis = 1)
full_data = full_data.round(2)
#Saving the data to a csv
full_data.to_csv('data_to_plot/USWages.csv', index = False)

full_data.head()

,degree_name,number_of_people,wage/salary,non-degree jobs,looking,looking_%,NDJ_%
0,General Agriculture,10170,64601.69,454,163,1.60,4.46
1,Agriculture Production And Management,7770,68821.81,431,159,2.05,5.55
2,Agricultural Economics,3366,88461.45,114,49,1.46,3.39
3,Animal Sciences,10352,64785.17,349,214,2.07,3.37
4,Food Science,2514,77567.30,88,86,3.42,3.50


# Data is now now ready for further analysis/visualization

CSVs have been created to include all the information to be further analysed in our Data Visualization Notebook. The CSV's will be used to create graphs.<br>
This information collected for each degree<br>

### USEduWages.csv
degree_name : name of degree<br>
number_of_people : number of people with that degree<br>
wage/salary : wages or salary income past 12 months<br>
non-degree jobs: number of people working in a job not in their field<br>
looking : people looking for jobs <br>
looking_% : percentage of people looking for jobs<br>
NDJ_% : percentage of people working in a job not in their degree<br>

### USOccuWages.csv
occupation_field : abbreviation of the field of the occupation<br>
occupation_name : name of occupation<br>
number_of_people : number of people with that degree<br>
wage/salary : wages or salary income past 12 months<br>

### USSexDiffWages.csv
degree_name	: name of degree<br>
male_% : percentage of male persons<br>
male_wage : count of male persons<br>
female_% : percentage of female persons<br>
female_wage : count of female persons <br>

## Please view visualization notebook